In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim

### Greedy fit

In [134]:
Nx = 2
Ne = 8
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
# Initialize map with zero coefficients
Hk = HermiteMapk(2, 3; α = 1e-6);

In [124]:
x = randn(2,1)
@show x
evaluate(Hk.I, x)

x = [-1.7385528410347308; -0.9202056157085414]


1-element Array{Float64,1}:
 -0.9202056157085414

In [125]:
reduced_margin = getreducedmargin(getidx(Hk))
idxnew = vcat(getidx(Hk), reduced_margin)

3×2 Array{Int64,2}:
 0  0
 0  1
 1  0

In [132]:
@time vcat(S.ψoff, zeros(1, 5))

  0.000009 seconds (3 allocations: 608 bytes)


9×5 Array{Float64,2}:
 1.0  1.0  0.620434   0.165862  0.620434
 1.0  1.0  0.610931   0.222977  0.610931
 1.0  1.0  0.430862  -0.532946  0.430862
 1.0  1.0  0.220301  -0.45219   0.220301
 1.0  1.0  0.626383  -0.114293  0.626383
 1.0  1.0  0.605263   0.249917  0.605263
 1.0  1.0  0.383338   0.54178   0.383338
 1.0  1.0  0.545307   0.418058  0.545307
 0.0  0.0  0.0        0.0       0.0

In [5]:
Nx = 2
Ne = 8
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';
B = MultiBasis(CstProHermite(3; scaled =true), Nx)

idx = [0 0; 0 1; 1 0; 2 1; 1 2]
truncidx = idx[1:2:end,:]
Nψ = 5

coeff =  [0.6285037650645056;
 -0.4744029092496623;
  1.1405280011620331;
 -0.7217760771894809;
  0.11855056306742319]
f = ExpandedFunction(B, idx, coeff)

fp = ParametricFunction(f)
R = IntegratedFunction(fp)

H = HermiteMapk(R; α = 1e-2)


S = Storage(H.I.f, ens.S);
addedidx = [2 2; 2 3; 2 4; 3 2]
@time update_storage!(S, ens.S, addedidx)

In [133]:
S.dψxd

8×5 Array{Float64,2}:
 0.0  -0.270855   0.0  -0.270855   -0.00861698
 0.0  -0.17488    0.0  -0.17488     0.469807
 0.0  -0.0775105  0.0  -0.0775105   0.602561
 0.0  -0.234487   0.0  -0.234487    0.297485
 0.0  -0.125779   0.0  -0.125779    0.552583
 0.0  -0.247966   0.0  -0.247966    0.235664
 0.0   0.234869   0.0   0.234869    0.295925
 0.0   0.246497   0.0   0.246497    0.243216

### Expanded Function

In [85]:
X = deepcopy(ens.S)

xk = X[Nx,:];

In [92]:
negative_log_likelihood!(S, H, ens.S)(nothing, zeros(Nψ), randn(Nψ))

In [91]:
typeof(10^5)

Int64

In [88]:
coeff = zeros(Nψ);

In [67]:
S = Storage(H.I.f, ens.S);

In [104]:
res = Optim.optimize(Optim.only_fg!(negative_log_likelihood!(S, H, X)), coeff, Optim.BFGS())
Optim.minimizer(res)

5-element Array{Float64,1}:
  0.550365898298582
 -0.8235761760320128
 -0.9742699195449483
  1.5527945828564937
  0.41476431680122144